In [1]:
import os
import ast
import json
import nltk
import time
import spacy
import openai
import random
import jellyfish
import numpy as np
import pandas as pd
from umap import UMAP
import seaborn as sns
from datetime import date
from bertopic import BERTopic
from nltk.tokenize import WhitespaceTokenizer
from bertopic.vectorizers import ClassTfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import matplotlib.pyplot as plt
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

C:\Users\kshit\anaconda3\envs\tf\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\kshit\anaconda3\envs\tf\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\kshit\anaconda3\envs\tf\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# jsonl -> dataframe

In [2]:
def jsonl_to_phdf(client,jsonl_name): #'P1_all.jsonl'
    with open(jsonl_name, 'r', encoding="utf8") as json_file:
        json_list = list(json_file)
        
    jos=[]
    for i in range(len(json_list)):
        jsonObject = json.loads(json_list[i])
        jos.append(jsonObject)
    
    cl = {'P4':'entities','P1':'label'}
    
    jos = pd.DataFrame(jos)
    jos = jos[jos[cl[client]].apply(lambda x:len(x))!=0]
    jos['nl'] = jos[cl[client]].apply(lambda x:len(x))
    jos = jos.sort_values(['id','nl']).drop_duplicates(['id'],keep='last')

    resultList = []
    for i in jos.index:
        jsonObject = jos.loc[i]
        for start, end, category in jos[cl[client]][i]:
            result = {}
            result["Review"] = jos["text"][i]
            if client=='P1':
                result["Review ID"] = str(jos["id"][i])
                result["Date"] = jos["date"][i]
                result["Rating"] = int(jos["rating"][i].split()[0])
            elif client=='P4':
                result["Review ID"] = 'P4R'+str(jos["id"][i])
                result["Date"] = jsonObject["Date"]
                result["Rating"] = int(jsonObject["Rating"].split()[0])
            result["Aspect"] = category.split("_")[0].strip()
            if category.split("_")[1].lower() == "neg":
                result["Sentiment"] = "Negative"
                result["Phrase"] = jos["text"][i][start:end].strip()
            elif category.split("_")[1].lower() == "pos":
                result["Sentiment"] = "Positive"
                result["Phrase"] = jos["text"][i][start:end].strip()
            resultList.append(result)

    df = pd.DataFrame.from_dict(resultList, orient='columns')
    df.drop_duplicates(inplace=True)
    return df

In [3]:
df1 = jsonl_to_phdf(client='P1',jsonl_name='P1_all.jsonl')

In [109]:
df = pd.read_csv('combined_df_20230105.csv')
df.drop_duplicates(inplace=True)
df.columns = ['Review ID','Review','Aspect','Sentiment','Phrase']
df = sampling(df,method='manual',value=350)
df['Aspect'] = df['Aspect'].apply(lambda x:str(x.capitalize()))
df['Sentiment'] = df['Sentiment'].astype(str)
df['Sentiment'] = df['Sentiment'].apply(lambda x:str(x.capitalize()))

# Sampling

In [106]:
def sampling(df,value=100,method='mean',stdev=0):
    df['cnt'] = df.groupby(['Review ID']).transform("count")['Phrase']
    
    if method=='mean':
        x = round(len(df)/df.Aspect.nunique()+stdev*np.std(df.Aspect.value_counts()))
    elif method=='manual':
        x = value
        
    l = []
    for aspect in df.Aspect.unique():
        if len(df[df.Aspect==aspect])>x:
            l.append(aspect)

    dfs = []
    cnt = df['cnt'].min()
    for aspect in l:
        temp_cnt=cnt
        while len(df[(df.Aspect==aspect)&(df.cnt<=temp_cnt)])<x:
            temp_cnt+=1
    #     print(aspect,temp_cnt,len(df[(df.Aspect==aspect)&(df.cnt<=temp_cnt)]))
        dfs.append(df[(df.Aspect==aspect)&(df.cnt<=temp_cnt)].sample(n=x,random_state=1))

    df = df[~df['Aspect'].isin(l)]
    for i in dfs:
        df = df.append(i)

    df['Aspect'] = df['Aspect'].astype(str)
    return df

In [19]:
df['cnt'] = df.groupby(['Review ID']).transform("count")['Phrase']

cvg = df[(df.Aspect=='Coverage')&(df.cnt<=3)].sample(n=99,random_state=1)
ti = df[(df.Aspect=='Time')&(df.cnt<=4)].sample(n=99,random_state=6)
fi = df[(df.Aspect=='Finish')&(df.cnt<=3)].sample(n=99,random_state=12)
sk = df[df.Aspect=='Skin'].sample(n=99,random_state=1)

df = df[~df['Aspect'].isin(['Coverage','Time','Finish','Skin'])]
df = df.append(cvg)
df = df.append(ti)
df = df.append(fi)
df = df.append(sk)
df['Aspect'] = df['Aspect'].astype(str)

for i in df.index:
    if df['Aspect'][i]=='Comp':
        df['Aspect'][i] = 'Competitor'
    if df['Aspect'][i]=='Demo':
        df['Aspect'][i] = 'Demographic'
df.drop(219,inplace=True)

# dataframe -> txt file

In [116]:
def phdf_to_txt(df):
    s = ''
    for id in df['Review ID'].unique():
    #     s = s + '##'+id+'\n'
        s = s + '##'
        s = s + list(df[df['Review ID']==id]['Review'])[0]+'\n\n\n'
        r = random.randint(1,10)
        if r%2==0:
            senti = ['Positive','Negative']
        else:
            senti = ['Negative','Positive']

        for sentiment in senti:
            s = s+sentiment.capitalize()+': \n'
            df1 = df[(df['Review ID']==id)]
            if len(df1[df1.Sentiment==sentiment])==0:
                s = s+'None\n'
            else:
                df1 = df1[(df1.Sentiment==sentiment)]
                for aspect in df1['Aspect'].unique():
                    s = s + aspect.capitalize()+': '
                    l = list(df1[df1.Aspect==aspect]['Phrase'])
                    if len(l)==1:
                        s = s + l[0] + '\n'
                    else:
                        for phrase in l:
                            s = s + phrase + ', '
                        s = s[:-2]
                        s+='\n'
            if sentiment==senti[0]:
                s = s+'\n'

    text_file = open("data.txt", "w",encoding="utf8") 
    text_file.write(s)
    text_file.close()

In [134]:
phdf_to_txt(df)

# txt -> jsonl

In [120]:
def txt_to_jsonl(jsonl_name): #"gpt3_train_20230111v1.jsonl"
    txt = open(r"data.txt","r",encoding="utf8")
    s = txt.read()

    #string cleaning
    while(s[0]!='#'):
        s = s[1:]

    l = s.split('##')
    l.pop(0)

    prompt = []
    completion = []

    for i in range(len(l)):
        prompt.append(l[i].split('\n\n\n')[0])
        completion.append(l[i].split('\n\n\n')[1])

    data = pd.DataFrame(list(zip(prompt, completion)), columns =['prompt', 'completion'])
    data['prompt'] = ' ' + data['prompt'] + '\n\n###\n\n'
    data['completion'] = ' ' + data['completion'] + ' END'

    data.to_json(jsonl_name,orient='records',lines=True)

In [146]:
txt_to_jsonl('gpt3_train_20230117v1.jsonl')

# Summary Generation

In [29]:
def summary(df,xlsx_name): #"1428_phrases_summary.xlsx"
    x = pd.DataFrame(df.groupby(['Aspect','Sentiment']).count()['Phrase'])
    x.columns = ['phrase_count']
    y = df.groupby(['Aspect','Sentiment']).mean()
    y.columns = ['phrase_avg_rating']

    final_df = x.join(y)
    final_df = final_df.reset_index()

    final_df['review_avg_rating']=''
    final_df['review_count']=''
    for aspect in final_df.Aspect.unique():
        for senti in ['Negative','Positive']:
            for i in final_df[(final_df['Aspect']==aspect)&(final_df['Sentiment']==senti)].index:
                final_df['review_count'][i]=df[(df['Aspect']==aspect)&(df['Sentiment']==senti)].Review.nunique()
                final_df['review_avg_rating'][i]=df[(df['Aspect']==aspect)&(df['Sentiment']==senti)].drop_duplicates(['Review ID']).mean()[0]
    final_df.to_excel(xlsx_name,index=False)            

# Fine Tuning GPT3

In [41]:
df = pd.read_csv('P4 First Aid Beauty.csv')
df = df[:300]
df.fillna('',inplace=True)
df['prompt'] = ' '+df['Review Title']+'\n\n'+df['Review']+'\n\n###\n\n'
df['completion']=''

In [13]:
API_KEY = "ORG = "os.environ['OPENAI_API_KEY'] = API_KEY
openai.api_key = os.getenv("OPENAI_API_KEY")

In [147]:
#Data Preparation Step
!openai tools fine_tunes.prepare_data -f gpt3_train_20230117v1.jsonl

^C


In [ ]:
#Fine Tuning Step
!openai api fine_tunes.create -t "gpt3_train_20230111v2.jsonl" -m "davinci"

In [42]:
for i in range(len(df)):
    prompt = df['prompt'][i]
    response = openai.Completion.create(
      model="davinci:ft-moksh-2023-01-11-15-14-35",
      prompt=prompt,
      temperature=0.7,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
      stop=["\n END"]
    )
    df['completion'][i] = response['choices'][0]['text']
    print(i,df['completion'][i])

0  Positive: 
Odor: pleasant

Negative: 
Value: raised the price
Ingredients: catastrophic change
1  Negative: 
None

Positive: 
Purchase: wish I bought more
Skin: winter dryness
2  Negative: 
Ingredients: eucalyptus
Finish: didn't leave me moisturized

Positive: 
None
3  Positive: 
None

Negative: 
Value: $78 for a 14oz jar?
Purchase: I cancelled it
4  Positive: 
Emotion: love this moisturizer

Negative: 
None
5  Negative: 
None

Positive: 
Purchase: must-have product
Application: works well
Time: long-lasting results
6  Positive: 
Application: feels so nice!
Value: worth the price
Skin: smoother my skin got

Negative: 
None
7  Negative: 
None

Positive: 
Purchase: must have in winter
Family: whole family
Time: during winter time
8  Positive: 
None

Negative: 
Emotion: honestly this stuff is not as moisturizing
Value: rip off
Application: makeup pill like crazy
Skin: retinol irritation
9  Negative: 
Value: basically cerave

Positive: 
None
10  Negative: 
None

Positive: 
Purchase: alw

95  Positive: 
None

Negative: 
Ingredients: reformulated
Emotion: doesn’t feel the same
96  Negative: 
Purchase: made me breakout
Skin: acne prone skin

Positive: 
None
97  Positive: 
Emotion: love it

Negative: 
None
98  Negative: 
Application: face break out
Skin: my entire face break out

Positive: 
None
99  Positive: 
Skin: hydrated without making me oily

Negative: 
None
100  Positive: 
Brand: love this fab moisturizing cream
Purchase: will continue to purchase
Skin: not sticky

Negative: 
None
101  Negative: 
Emotion: favorite moisturizer

Positive: 
Time: sinks in as fast
102  Positive: 
None

Negative: 
Design: feels like a little too much
Skin: normal sensitive skin
103  Negative: 
Odor: petro chemical fishy mold
Emotion: moving on to a more reliable

Positive: 
None
104  Positive: 
None

Negative: 
Purchase: not hydrating at all
Skin: my skin feels so dry
Competitor: Farmacy Halo Honey and Dewy Skin
105  Positive: 
Competitor: “water bank” by Tatcha

Negative: 
None
106  Pos

186  Positive: 
Skin: sensitive skin

Negative: 
None
187  Negative: 
Ingredients: eucalyptus

Positive: 
None
188  Positive: 
Motivation: better one without silicones

Negative: 
Ingredients: dimethicone
189  Positive: 
Emotion: incredible product
Skin: sensitive skin

Negative: 
None
190  Positive: 
Purchase: go-to moisturizer
Skin: irritated, cracked skin

Negative: 
None
191  Positive: 
Demographic: I use this morning and night

Negative: 
None
192  Negative: 
None

Positive: 
Emotion: favorite moisturizer
Purchase: worth the purchase
193  Negative: 
None

Positive: 
Value: lucky find
Finish: plump without being greasy
194  Negative: 
None

Positive: 
Skin: nourish the skin
Ingredients: great ingredients
Application: sinks in well
195  Positive: 
Application: nice cooling affect
Skin: dry skin feel moisturized

Negative: 
None
196  Negative: 
None

Positive: 
Value: worth the price tag
Skin: rough areas
197  Positive: 
None

Negative: 
Purchase: Won’t Be Repurchasing
Ingredients: p

283  Positive: 
Emotion: I love this moisturizer

Negative: 
None
284  Positive: 
Value: huge amount for the price
Time: timeless

Negative: 
None
285  Negative: 
None

Positive: 
Moisturizer: saved me
Time: put it on at night
286  Negative: 
Application: takes more effort than most of my body products to work into the skin and fully dissolve
Value: pretty thick and takes more effort than most of my body products to work into the skin and fully dissolve, pretty expensive

Positive: 
Ingredients: some really great ingredients in here that are genuinely good for skin health
Skin: heal and soothe skin
287  Negative: 
Ingredients: felt like it was just there
Skin: burned

Positive: 
None
288  Positive: 
Emotion: so good!
Skin: sensitive skin

Negative: 
None
289  Positive: 
Purchase: purchased my second tub!!
Skin: plumpness

Negative: 
None
290  Negative: 
Motivation: heard so many good things
Skin: my face gets irritated

Positive: 
None
291  Positive: 
Time: skin feeling so good!

Negat

In [43]:
df = process_completion(df)
df = completion_to_phdf(df)

# completion processing

In [21]:
def process_completion(df):
    df['Negative']=''
    df['Positive']=''

    for i in df.index:
        if type(df['completion'][i])==float:
            continue
        l1 = df['completion'][i].split('###')
        if len(l1)>=2:
            l = df['completion'][i].split('###')[1].split('\n\n')
        else:
            l = df['completion'][i].split('\n\n')
        if(len(l)==1):
    #         print(i,df['completion'][i],l)
            continue
        if l[0]=='':
            l.pop(0)
    #     if l[0][1]=='#':
    #         l[0] = l[0][3:]
        l[0] = l[0].strip()
    #     print(i)
        try:
            if l[0][0].lower()=='n':
                df['Negative'][i] = l[0]
                df['Positive'][i] = l[1]
            else:
                df['Negative'][i] = l[1]
                df['Positive'][i] = l[0]
        except:
            pass

    df['Negative'] = df['Negative'].str.split('Negative: ',expand=True)[1]
    df['Positive'] = df['Positive'].str.split('Positive: ',expand=True)[1]
    df['Negative'] = df['Negative'].str.split('\n')
    df['Positive'] = df['Positive'].str.split('\n')
    return df

# completion -> phrase dataframe

In [28]:
def completion_to_phdf(df): #'P1_Estee Lauder.csv'
#     daf = pd.read_csv('P1_Estee Lauder.csv')
#     df = pd.merge(df,daf.loc[:,['Review ID','Date','Rating']],on='Review ID')
#     df.drop_duplicates(['Review ID'],inplace=True)
#     df['Rating'] = df["Rating"].apply(lambda x : int(x.split()[0]))
#     df['Date'] = pd.to_datetime(df['Date'])

    resultlist=[]
    for i in df.index:
        for senti in ['Positive','Negative']:
            if df[senti][i] is None or type(df[senti][i])==float:
                continue
                
            if type(df[senti][i])==list:
                l = df[senti][i]
            else:
                l = ast.literal_eval(df[senti][i])
                
            for aspects in l:
                aspect = aspects.split(':')[0]
                try:
                    phrases = aspects.split(':')[1]
                except:
                    continue
                for phrase in phrases.split(','):
                    result={}
                    result['Review ID'] = df['Review ID'][i]
                    result['Sentiment'] = senti
                    result['Review'] = df['Review'][i]
                    result['Rating'] = df['Rating'][i]
                    result['Date'] = df['Date'][i]
                    result['Phrase'] = phrase
                    result['Aspect'] = aspect
                    resultlist.append(result)
    df3 = pd.DataFrame.from_dict(resultlist, orient='columns')
    df3['Rating'] = df3["Rating"].apply(lambda x : int(x.split()[0]))
    df3['Date'] = pd.to_datetime(df3['Date'])
    df3.drop_duplicates(inplace=True)
    df3 = df3.sort_values(['Sentiment','Aspect'],ascending=False)
    df3 = df3[df3['Phrase'].astype(bool)]
    df3.index = range(len(df3))
    return df3

# Detailed Output

In [2]:
df = pd.read_csv('P4_combined_df_20230105.csv')

In [3]:
dP4 = {'Performance':['Allergy','Performance','Skin'],
         'Application':['Application','Texture'],
        'Emotional':['Brand','Competitor','Purchase','Sentiment'],
        'Product':['Demographic','Motivation','Odor','Packaging','Price']}

In [4]:
dP1 = { 'Performance':['Skin','Performance'],
       'Application':['Texture','Application'],
       'Emotional':['Brand','Competitor','Purchase'],
    'Product':['Value','Time','Partner','Odor','Motivation','Ingredients','Design',
                  'Finish','Emotion','Demographic','Coverage','Color']}

In [91]:
l = detailed_output(df,map_dict=dP4,client='P4',ng_range=(1,3),words_per_topic=5,compress=True,create_exl=False,version='v1')
#with distilbert-base-cased

Clustering Started

BERTopic Clustering will take a while
Sentiment: Positive Aspect: Ingredients
   Topic  Count                                            Name
0      0     42  0_ingredient_clean_ingredient ingredient_great
1      1     15       1_colloidal_colloidal oatmeal_oatmeal_oat
            Cluster
ingredient       42
colloidal        15
      Cluster    L3 Cluster Phrase
0   colloidal    colloidal oatmeal
1  ingredient  ove the ingredients 

Sentiment: Positive Aspect: Brand
   Topic  Count                              Name
0      0     51          0_fab_brand_product_love
1      1     19  1_aid_first_first aid_aid beauty
     Cluster
fab       51
aid       19
  Cluster        L3 Cluster Phrase
0     fab           ove this brand
4     aid  Thanks First Aid Beauty 

Sentiment: Positive Aspect: Purchase
   Topic  Count                                               Name
0      0     90                       0_buy_purchase_size_continue
1      1     20  1_repurchase_repurchase r

         Cluster    L3 Cluster Phrase
0           acne           treat acne
1         eczema          have eczema
2           hand       very dry hands
3            dry         skin is dry,
4      sensitive            sensitive
6        rosacea      ered my rosacea
7        redness       reduce redness
8     acne prone      acne prone skin
19     dry patch          dry patches
21    dermatitis  perioral dermatitis
26         itchy      wintery itching
38       dryness  helped with dryness
41          soft      my skin so soft
44         crack      red and cracked
60      moisture    locks in moisture
72          skin     repaired my skin
80   moisturizer  ultra moisturizing!
98    irritation  reducing irritation
128         rash        horrible rash
253       winter     dry winter skin. 

Sentiment: Positive Aspect: Motivation
   Topic  Count                                  Name
0      0     38  0_sephora_moisturizer_skin_recommend
1      1     30        1_review_read_good_good review

In [78]:
l = detailed_output(df,map_dict=dP4,client='P4',ng_range=(1,3),words_per_topic=5,compress=False,create_exl=False,version='v1')
#Without distilbert-base-cased

Clustering Started

BERTopic Clustering will take a while
Sentiment: Positive Aspect: Ingredients
   Topic  Count                                            Name
0      0     42  0_ingredient_clean_ingredient ingredient_great
1      1     15       1_colloidal_colloidal oatmeal_oatmeal_oat
            Cluster
ingredient       42
colloidal        15
      Cluster    L3 Cluster Phrase
0   colloidal    colloidal oatmeal
1  ingredient  ove the ingredients
Sentiment: Positive Aspect: Brand
   Topic  Count                              Name
0      0     51          0_fab_brand_product_love
1      1     19  1_aid_first_first aid_aid beauty
     Cluster
fab       51
aid       19
  Cluster        L3 Cluster Phrase
0     fab           ove this brand
4     aid  Thanks First Aid Beauty
Sentiment: Positive Aspect: Purchase
   Topic  Count                                               Name
0      0     90                       0_buy_purchase_size_continue
1      1     20  1_repurchase_repurchase repur

             Cluster   L3 Cluster Phrase
0     skin sensitive      sensitive skin
1           dry skin        ery dry skin
8          extremely  extremely dry skin
9              combo     combination ski
15     dry sensitive  dry sensitive skin
30              oily           oily skin
54  combination skin    combination skin
Sentiment: Positive Aspect: Skin
    Topic  Count                                               Name
0       0     75                         0_acne_pore_clog_clog pore
1       1     54  1_eczema_eczema eczema_eczema eczema eczema_ke...
2       2     39                2_dry dry_dry_mostly_dry mostly dry
3       3     38                         3_dry patch_patch_dry_spot
4       4     34      4_acne prone_prone_acne prone skin_prone skin
5       5     31  5_rosacea_rosacea rosacea_calm rosacea_rosacea...
6       6     31            6_flaky_flake_flaky skin_dry flaky skin
7       7     30           7_redness_calm redness_calm_redness calm
8       8     42      8_ecz

Sentiment: Negative Aspect: Price
   Topic  Count                                       Name
0     -1     34  -1_size expect_price oz_opinion_love pric
             Cluster
size expect       34
       Cluster    L3 Cluster Phrase
0  size expect  not worth the price
Sentiment: Negative Aspect: Skin_type
   Topic  Count                                              Name
0      0     25                      0_dry_dry skin_skin_skin dry
1      1     25  1_sensitive_sensitive skin_skin_combination skin
2      2     16         2_oily_oily skin_skin oily_oily skin oily
           Cluster
dry             25
sensitive       25
oily            16
     Cluster   L3 Cluster Phrase
0       oily  not for oily skin!
1        dry       very dry skin
3  sensitive      sensitive skin
Sentiment: Negative Aspect: Skin
   Topic  Count                             Name
0      0    132            0_skin_dry_acne_prone
1      1     12  1_pore_clog_clog pore_pore clog
      Cluster
skin      132
pore       12
  

In [90]:
def detailed_output(df,map_dict,client,flag_rating=4,compress=True,words_per_topic=5,ng_range=(1,3),
                    date=date.today().strftime('%Y%m%d'),version='v1',create_exl=False):
    df3=df
    #########################################sheet5##########################################
    sheet5 = pd.DataFrame(columns=['Sentiment','L1 Cluster ID','L1 Cluster','L1 Phrase Count','L1 Review Count',
                                   'L1 Rating (Phrase)','L1 Rating (Review)','L2 Cluster ID','L2 Cluster','L2 Phrase Count',
                                   'L2 Review Count','L2 Rating (Phrase)','L2 Rating (Review)','L3 Cluster ID','L3 Cluster',
                                   'L3 Cluster Phrase','L3 Phrase Count','L3 Review Count','L3 Rating (Review)',
                                   'L3 Rating (Phrase)','L4 ID','L4 Phrases','Review ID','Review Rating'])
    sheet5['Sentiment'] = df3.Sentiment
    sheet5['L2 Cluster'] = df3.Aspect
    sheet5['L4 Phrases'] = df3.Phrase
    sheet5['Review ID'] = df3['Review ID']
    sheet5['Review Rating'] = df3.Rating
    sheet5['Date'] = pd.to_datetime(df3.Date)
#     sheet5 = sheet5.merge(daf.loc[:,['Review ID','Verified Purchase', 'Recommended',
#            'Incentivized', 'Upvote', 'Downvote', 'AgeRange', 'SkinConcerns']],on='Review ID')
    sheet5.fillna('',inplace=True)
    print("Clustering Started\n")
    
    #######################################Clustering###########################################
    print("BERTopic Clustering will take a while")
    nlp = spacy.load("en_core_web_sm")
    stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your',
     'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 
    'its','itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 
    'these','those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did',
    'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 
    'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down',
    'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how',
    'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'nor', 'only', 'own', 'same', 'so', 'than', 'too',
    'very', 's', 't','can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 
    'ain', 'aren', "aren't",'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven',
     "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', 
     "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't", 'since','first', 'even',
     'thisssss', 'could','really','always','bc','truly','literally','48','none','yet','actually','like','would','ever','issss',
    'although','much','10','do','super','c','f','definitely','completely','totally','ski','ive','not'] # 'not','no'
    def tokenization(text):
        tk = WhitespaceTokenizer()
        return tk.tokenize(text)
    def remove_stopwords(text):
        output= [i for i in text if i not in stopwords]
        return output
#         ps = SnowballStemmer("english")

    df3.dropna(inplace=True)
#     v = CountVectorizer(ngram_range = ng_range,stop_words="english")
#     vd = v.fit_transform(list(df3.loc[:, "Phrase"].values))
    
    start_time = time.time()
    for senti in ['Positive','Negative']:
        for aspect in df3.Aspect.unique():
            print('Sentiment:',senti,'Aspect:',aspect)
            df1 = df3[(df3.Aspect==aspect)&(df3.Sentiment==senti)]
            df1 = df1.reset_index()
            if len(df1)==0:
                print("There are no phrases for this aspect & sentiment.")
                continue

            docs = list(df1.loc[:, "Phrase"].values)

            # Lemmatization
            for i in range(len(docs)):
                doc = nlp(docs[i])
                tokens = []
                for token in doc:
                    tokens.append(token)
                lemmatized_sentence = " ".join([token.lemma_ for token in doc])
                docs[i] = lemmatized_sentence

            docs = pd.Series(docs).apply(lambda x: tokenization(x))
            docs = docs.apply(lambda x:remove_stopwords(x))
            for i in range(len(docs)):
                docs[i] = " ".join(docs[i])

            lst = ['ADV','VERB','ADJ','NOUN','PROPN','X'] #,'ADV'

            for i in range(len(docs)):
                sent = nlp(docs[i])
                text= []
                for word in sent:
                    if not word.is_punct and word.pos_ in lst:#not word.is_stop and
                        text.append(word.lemma_) #ps.stem(word.lemma_)
                
                suffix = {'hydrating':'hydrate','hydrated':'hydrate','hydration':'hydrate'}
                for j in range(len(text)):
                    for su,w in suffix.items():
                        if text[j].endswith(su):
                            text[j] = text[j][:-len(su)] + w
                            
                k = []
                st = ' '.join(text)
                for x in text:
                    if (st.count(x)>=1 and (x not in k)):
                        k.append(x)
#                 k.sort()
                docs[i] = ' '.join(k)

            docs = pd.Series(docs)
#                 print(docs)
            df1['cp'] = docs
            df1 = df1[df1.cp.astype(bool)]
            docs = docs[docs.astype(bool)]
            df1.index=range(len(df1))
            docs.index=range(len(docs))
            
            if compress:
                compress="auto"
            else:
                compress=None
                
            try:
                modal = BERTopic(#embedding_model=embedding_model,
                             #hdbscan_model=hdbscan_model,
                             top_n_words = words_per_topic,
                             calculate_probabilities = True,
                             vectorizer_model = TfidfVectorizer(
#                                              vocabulary = v.get_feature_names_out(),
                                             ngram_range = ng_range,
                                             stop_words="english"),
                             nr_topics=compress,
                             diversity=1,
                             umap_model=UMAP(random_state=2),
                             ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True,bm25_weighting=True),
                             language="english",
                             n_gram_range=ng_range)
                topics, probs = modal.fit_transform(docs.values)
                
                try:
                    topics = modal.reduce_outliers(docs.values, topics, probabilities=probs, strategy="probabilities")
#                     topics = modal.reduce_outliers(docs,topics,strategy="embeddings")
                    modal.update_topics(docs.values, topics=topics)
                except:
                    pass
                
                df1['Cluster']=''
#                     predicted_topics, predicted_probs = modal.transform(docs)
                for i in range(len(df1)):
                    s = modal.get_topic(topics[i])[0][0]
                    l = s.split()
                    k = []
                    for x in l:
                        if (s.count(x)>=1 and (x not in k)):
                            k.append(x)
                    df1['Cluster'][i] = ' '.join(k)
#                 fig = modal.visualize_heatmap()
#                 fig.show()
#                 filename = senti+"_"+aspect+".html"
#                 fig.write_html(filename)
                print(modal.get_topic_info())
                print(pd.DataFrame(df1['Cluster'].value_counts()))
    #                 print(modal.get_document_info(docs.values))
            except Exception as e:
                print(e)
                print('Aspect contains insufficient number of phrases for clustering. BERTopic Error.\n')
                df1['Cluster']=aspect
            
            df1['L3 Cluster Phrase']=''
            df1['sim'] = ''
            df1.drop(['index'],1,inplace=True)
            if len(df1)==1:
                df1['L3 Cluster Phrase'] = df1['Phrase'].iloc[0]
            else:
                for cluster in df1['Cluster'].unique():
                    for i in df1[df1.Cluster==cluster].index:
                        sim=[]
                        for j in df1[df1.Cluster==cluster].index:
                            if i!=j:
                                sim.append(jellyfish.levenshtein_distance(df1['Phrase'][i],df1['Phrase'][j]))
                        df1['sim'][i] = sum(sim)/len(sim)
                        
                    m = df1[df1.Cluster==cluster]['sim'].min()
                    p = list(df1[(df1.Cluster==cluster)&(df1['sim']==m)]['Phrase'])[0]
                    for i in df1[df1.Cluster==cluster].index:
                        df1['L3 Cluster Phrase'][i] = p
            print(df1.loc[:,['Cluster','L3 Cluster Phrase']].drop_duplicates(),'\n')
            
            cluster_count = {}
            for review in df1.Review.unique():
                for cluster in df1[df1.Review==review]['Cluster'].unique():
                    try:
                        cluster_count[cluster]+=1
                    except:
                        cluster_count[cluster]=1
            x = pd.DataFrame(df1.groupby('Cluster').count()['Phrase'])
            x.columns = ['L3 Phrase Count']
            y = pd.DataFrame(cluster_count.values(),index=cluster_count.keys(),columns=['L3 Review Count'])
            z = pd.DataFrame(df1.groupby('Cluster').mean()['Rating'])
            z.columns = ['L3 Rating (Phrase)']
            a =pd.DataFrame(df1.groupby(['Cluster','Review']).mean().groupby('Cluster').mean()['Rating'])
            a.columns = ['L3 Rating (Review)']
            layer3 = x.join(y).join(z).join(a)
            layer3['Cluster'] = layer3.index
            layer3.index=range(len(layer3))
            
            s = sheet5[(sheet5['L2 Cluster']==aspect)]
            for i in s[s.Sentiment==senti].index:
                try:
                    c = list(df1[df1.Phrase==sheet5['L4 Phrases'][i]]['Cluster'])[0]
                    sheet5['L3 Cluster'][i] = list(layer3[layer3.Cluster==c]['Cluster'])[0]
                    sheet5['L3 Cluster Phrase'][i] = list(df1[df1.Phrase==sheet5['L4 Phrases'][i]]['L3 Cluster Phrase'])[0]
                    sheet5['L3 Phrase Count'][i] = list(layer3[layer3.Cluster==c]['L3 Phrase Count'])[0]
                    sheet5['L3 Review Count'][i] = list(layer3[layer3.Cluster==c]['L3 Review Count'])[0]
                    sheet5['L3 Rating (Phrase)'][i] = list(layer3[layer3.Cluster==c]['L3 Rating (Phrase)'])[0]
                    sheet5['L3 Rating (Review)'][i] = list(layer3[layer3.Cluster==c]['L3 Rating (Review)'])[0]
                except:
                    pass
    print("Clustering done after {:.2f} sec \n".format(time.time() - start_time))
    sheet5 = sheet5[sheet5['L3 Cluster'].astype(bool)]
    ###############################Layer2##############################################
    for senti in ['Positive','Negative']:
        a = sheet5[sheet5.Sentiment==senti]
        L2 = {}
        for aspect in a['L2 Cluster'].unique():
            b = a[a['L2 Cluster']==aspect]
            d = {}
            for cluster in b['L3 Cluster'].unique():
                c = b[b['L3 Cluster']==cluster]
                pcount = list(b[b['L3 Cluster']==cluster]['L3 Phrase Count'])[0]
                rcnt = list(b[b['L3 Cluster']==cluster]['L3 Review Count'])[0]
                prating = list(b[b['L3 Cluster']==cluster]['L3 Rating (Phrase)'])[0]
                rrat = list(b[b['L3 Cluster']==cluster]['L3 Rating (Review)'])[0]
                d[cluster] = [pcount,prating,rcnt,rrat]
            p_total = 0
            p_product = 0
            r_total = 0
            r_product = 0
            for key,value in d.items():
                try:
                    p_total+=value[0]
                    r_total+=value[2]
                except:
                    continue
                p_product+=(value[0]*value[1])
                r_product+=(value[2]*value[3])
            try:
                L2[aspect] = [p_total,p_product/p_total,r_total,r_product/r_total]
            except:
                continue
        for aspect,value in L2.items():
            for i in sheet5[(sheet5.Sentiment==senti) & (sheet5['L2 Cluster']==aspect)].index:
                sheet5['L2 Phrase Count'][i] = value[0]
                sheet5['L2 Rating (Phrase)'][i] = value[1]
                sheet5['L2 Review Count'][i] = value[2]
                sheet5['L2 Rating (Review)'][i] = value[3]
    print("Layer 2 Completed\n")
    
    #############################################Layer1#########################################
    d = map_dict
    
    for i in sheet5.index:
        l1 = ''
        for key,values in d.items():
            if sheet5['L2 Cluster'][i] in values:
                l1 = key
                break
        sheet5['L1 Cluster'][i] = key

    # sheet5['L1 Rating (Phrase)'] = sheet5['L1 Rating (Phrase)'].astype(float)
    # sheet5['L1 Rating (Review)'] = sheet5['L1 Rating (Review)'].astype(float)

    for senti in ['Positive','Negative']:
        a = sheet5[sheet5.Sentiment==senti]
        L2 = {}
        for aspect in a['L1 Cluster'].unique():
            b = a[a['L1 Cluster']==aspect]
            d = {}
            for cluster in b['L2 Cluster'].unique():
                c = b[b['L2 Cluster']==cluster]
                pcount = list(b[b['L2 Cluster']==cluster]['L2 Phrase Count'])[0]
                rcnt = list(b[b['L2 Cluster']==cluster]['L2 Review Count'])[0]
                prating = list(b[b['L2 Cluster']==cluster]['L2 Rating (Phrase)'])[0]
                rrat = list(b[b['L2 Cluster']==cluster]['L2 Rating (Review)'])[0]
                d[cluster] = [pcount,prating,rcnt,rrat]
            p_total = 0
            p_product = 0
            r_total = 0
            r_product = 0
    #         print(d)
            for key,value in d.items():
#                 print(senti,aspect,key,value)
                try:
                    p_total+=value[0]
                    r_total+=value[2]
                except:
                    continue
                p_product+=(value[0]*value[1])
                r_product+=(value[2]*value[3])
            try:
                L2[aspect] = [p_total,p_product/p_total,r_total,r_product/r_total]
            except:
                continue
        for aspect,value in L2.items():
            for i in sheet5[(sheet5.Sentiment==senti) & (sheet5['L1 Cluster']==aspect)].index:
                sheet5['L1 Phrase Count'][i] = value[0]
                sheet5['L1 Rating (Phrase)'][i] = value[1]
                sheet5['L1 Review Count'][i] = value[2]
                sheet5['L1 Rating (Review)'][i] = value[3]
    print("Layer 1 Completed\n")
    
    ###################################Id insertion & output excel formation################################
    id = 1
    for u_phrase in sheet5['L4 Phrases'].unique():
        for i in sheet5[sheet5['L4 Phrases']==u_phrase].index:
            sheet5['L4 ID'][i] = client+'L4I'+ str(id)
        id+=1

    id = 1
    for ul2_cluster in sheet5['L2 Cluster'].unique():
        for ul3_cluster in sheet5[sheet5['L2 Cluster']==ul2_cluster]['L3 Cluster'].unique():
            for i in sheet5[(sheet5['L2 Cluster']==ul2_cluster) & (sheet5['L3 Cluster']==ul3_cluster)].index:
                sheet5['L3 Cluster ID'][i] = client+'L3I'+ str(id)
            id+=1

    id = 1
    for u_cluster in sheet5['L2 Cluster'].unique():
        for i in sheet5[sheet5['L2 Cluster']==u_cluster].index:
            sheet5['L2 Cluster ID'][i] = client+'L2I'+ str(id)
        id+=1

    id = 1
    for u_cluster in sheet5['L1 Cluster'].unique():
        for i in sheet5[sheet5['L1 Cluster']==u_cluster].index:
            sheet5['L1 Cluster ID'][i] = client+'L1I'+ str(id)
        id+=1

    d1 = sheet5
    d2 = sheet5.iloc[:,:20]
    d3 = sheet5.iloc[:,:13]
    d4 = sheet5.iloc[:,:7]
    
    d1 = d1.sort_values(['L1 Cluster','L2 Cluster','Sentiment','L3 Phrase Count','L3 Cluster'],ascending=False)
    d2 = d2.sort_values(['L1 Cluster','L2 Cluster','Sentiment','L3 Phrase Count','L3 Cluster'],ascending=False)
    d3 = d3.sort_values(['L1 Cluster','Sentiment','L2 Phrase Count','L2 Cluster'],ascending=False)
    d4 = d4.sort_values(['L1 Cluster','Sentiment'],ascending=False)

    d2.drop_duplicates(['Sentiment','L2 Cluster','L3 Cluster'],inplace=True)
    d3.drop_duplicates(['Sentiment','L2 Cluster'],inplace=True)
    d4.drop_duplicates(['Sentiment','L1 Cluster'],inplace=True)
    
    for x in [d1,d2]:
        x['Flag']=''
        for i in x.index:
            if x['L3 Rating (Review)'][i]>=flag_rating:
                x['Flag'][i] = x['Sentiment'][i][0]+'+'
            else:
                x['Flag'][i] = x['Sentiment'][i][0]+'-'
        
#     d2.replace('',)
#     d3.replace('',np.nan,inplace=True)
#     d3.dropna(inplace=True)
    if create_exl:
        name = client+"_detailed_"+date+version+".xlsx"
        path = r"{fname}".format(fname=name)
        with pd.ExcelWriter(path) as engine:
            d1.to_excel(excel_writer=engine,sheet_name='L4',index=False)
            d2.to_excel(excel_writer=engine,sheet_name='L3',index=False)
            d3.to_excel(excel_writer=engine,sheet_name='L2',index=False)
            d4.to_excel(excel_writer=engine,sheet_name='L1',index=False)
        print("Excel File with 4 sheets is created.\n")
        print("File name:",name)
    dfs = [d1,d2,d3,d4]
    return dfs